# 4 - Feras Formidáveis

## 4.9 A senha de tamanho variável

### Introdução:

O objetivo desta atividade é resolver o problema da senha em algoritmos genéticos sem forneçer a informação
do tamanho da senha para a função que gera a população, considerando que a senha pode ser uma string de 1 até 30 caracteres. Boa parte do código foi adaptada dos materiais de aula do Daniel Cassar [1-X], em que criou-se scripts específicos para o projeto, os quais serão comentados ao longo da resolução. O exercício foi realizado em conjunto com o Guilherme Sobreira, em que discutimos maneiras de modificar o algoritmo genético original.

### Desenvolvimento:

Importando as bibliotecas necessárias e os scripts [2-5]:

In [212]:
import random
from string import ascii_lowercase, ascii_uppercase, digits # importa as letras minúsculas, maiúsculas e dígitos
from Scripts import populacao_senha_size as cria_populacao
from Scripts import funcao_objetivo_pop_senha_size as funcao_objetivo
from Scripts import selecao_torneio_min as funcao_selecao
from Scripts import cruzamento_uniforme as funcao_cruzamento
from Scripts import mutacao_salto_tamanho as funcao_mutacao1
from Scripts import mutacao_simples as funcao_mutacao2
from Scripts import mutacao_salto as funcao_mutacao3

Definindo os hiperparâmetros do problema, dentre os quais criamos uma senha formada por uma lista com cada letra em string, além dos caracteres possíveis que formam uma senha, com letras maiúsculas, minúsculas e os dígitos

In [213]:
SENHA = list("Su1relyNotASecur6ePassword3")
CARACTERES_POSSIVEIS = ascii_lowercase + ascii_uppercase + digits

TAMANHO_POPULACAO = 100
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO_LETRA = 0.025
CHANCE_DE_MUTACAO_TAMANHO = 0.05
TAMANHO_TORNEIO = 3

Criando a população inicial do problema, em que cada indivíduo pode conter uma senha entre 1 e 30 caracteres:

In [214]:
populacao = cria_populacao(TAMANHO_POPULACAO, CARACTERES_POSSIVEIS)
#populacao

A função objetivo desse problema considera tanto a distância ordinal entre a letra do candidato e a letra da senha verdadeira, quanto a distância entre o tamanho do candidato e da senha. Aplicou-se um peso proporcional à diferença entre os extremos do ordinal para os caracteres possíveis, garantindo que o algoritmo encontre primeiro um canditado com o mesmo tamanho da senha verdadeira.

In [215]:
# Analisando o ordinal para os caracteres possíveis
print([letra for letra in CARACTERES_POSSIVEIS], '\n', [ord(letra) for letra in CARACTERES_POSSIVEIS])


['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] 
 [97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57]


In [216]:
lista_ord = list(ord(letra) for letra in CARACTERES_POSSIVEIS)
peso = max(lista_ord) - min(lista_ord) + 1
peso

75

O código principal, contendo um laço while para fazer todas as etapas do algoritmo genético (seleção, cruzamento, mutação e atualização da geração) até encontrar a senha correta. Curiosamente, aplicar um cruzamento uniforme funcionou para encontrar a senha correta, após tentativas falhas de criar uma função de cruzamento específica para este problema. Além disso, aplicar 3 funções de mutação diferentes demonstrou ser benéfico pro treinamento do algoritmo, primeiro podendo alterar o tamanho 

In [217]:
individuo = ['V', 'B', '1', 'n', 's', 'U', 'e']
gene = random.randint(0, len(individuo))
#gene = len(individuo) # 7
print(f'Indivíduo {individuo}, gene {gene}, caracter {individuo[gene]}')

if gene == 0:
    individuo.insert(0,random.choice(CARACTERES_POSSIVEIS))
elif gene == len(individuo):
    individuo.pop(gene-1)
else:
    # sorteia se vai adicionar ou remover o gene
    fator_transicao = random.randint(-1,1)
    if fator_transicao == -1:
        individuo.pop(gene)
    else:
        individuo.insert(gene,random.choice(CARACTERES_POSSIVEIS))
print(fator_transicao)
print(f'Indivíduo {individuo}, gene {gene}, caracter {individuo[gene]}')

Indivíduo ['V', 'B', '1', 'n', 's', 'U', 'e'], gene 5, caracter U
0
Indivíduo ['V', 'B', '1', 'n', 's', 'q', 'U', 'e'], gene 5, caracter q


In [218]:
menor_fitness_geral = float("inf")
geracao = 0

while menor_fitness_geral != 0:

    # Seleção
    fitness = funcao_objetivo(populacao, SENHA)
    selecionados = funcao_selecao(populacao, fitness, TAMANHO_TORNEIO)

    # Cruzamento
    proxima_geracao = []
    for pai, mae in zip(selecionados[::2], selecionados[1::2]):
        individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
        proxima_geracao.append(individuo1)
        proxima_geracao.append(individuo2)

    # Mutação
    funcao_mutacao1(proxima_geracao, CHANCE_DE_MUTACAO_TAMANHO, list(CARACTERES_POSSIVEIS))
    funcao_mutacao2(proxima_geracao, CHANCE_DE_MUTACAO_LETRA, list(CARACTERES_POSSIVEIS))
    funcao_mutacao3(proxima_geracao, CHANCE_DE_MUTACAO_LETRA, list(CARACTERES_POSSIVEIS))

    # Encerramento
    populacao = proxima_geracao
    geracao += 1

    fitness = funcao_objetivo(populacao, SENHA)
    menor_fitness_observado = min(fitness)

    if menor_fitness_observado < menor_fitness_geral:
        menor_fitness_geral = menor_fitness_observado
        indice = fitness.index(menor_fitness_observado)
        candidato = populacao[indice]
        print(geracao, "".join(candidato))

1 615RTXeNwv0cFNatC1t5kfUumlnY
2 FGDiXOJYnpeKkyWoKx4WF7OpQt9
3 FGDRTOeNwp0KkyaoCx4Wk7Upml9
4 6W5DiCiNpvZXkNWtCI4WkfUJmo9
5 QWDiiXiIpkKfkyWoKx4WFlOpQc9
8 6GEqpXoNwv50cnqoKtMEfxkfccM
9 QG5RoXoNwvk0ciqoKKMEtxlpcc9
10 QGEifXiYpkBffyqtKLEWflPpcc9
11 QW9iiXoNppKZcnyoKtKWFlxpQc9
12 QYEqiXoYpvBKkyqtCxMWflPppc9
13 QY9qiXoNppKZcnyoKxMEfxlpmc9
14 QG9qfXoYppBffqqt1tKWkllppc9
15 QYEqiXoNpvKKcyytCxMWtxxppc9
17 QY9qiXoYppBZfnqo1tKWkxlpmc9
19 QY9qiXoNppBZcnyo1tMWkxlppc9
21 QY9qiXoNppBZcnyo1tMWkxxppc9
22 QY9qiXoNpvBXfnyt1wMWtlxppc9
26 QY9qiXoNpvBXfnyt1tMWtxxpmc9
28 QY9qiyoNpvBXfnyo1tMWtxxpmc9
30 QY9qiyoNpvBXfnyt1tNWtxxppc9
31 QY8qiyoNpvBXfnyt1tMWtxxppc8
32 QY0qiyoNpvBXfnqo1wMWtxxppc8
33 QY9qiyoNqvBXfnyt1hNWtxxppc9
34 QY8qiyoNqvBWfnyt1hNWtxxppc9
35 QY0qiyoNqvBXfnyt1hNWtxxppc8
36 QY0qiyoNqvBWfnyt1hNWtxxppc8
38 QY0qiyoNpvBXenqp1hMWtxwppc8
39 QY0qiyoNpvBWfnyp1hNWtxwppc8
40 QY0qiyoNpvBWenqt1hNWtxwppc8
42 RY0qiyoNovBWfnyt1hNWtxwppc8
44 RY1qiyoNovBWenqt1hNWtxxppc8
45 RY0qhyoNovBWenqt1hNWtxwppc8
46 RY1qhyoNovB

### Conclusão:

Foi possível treinar uma rede neural em python puro para tarefas de classificação, embora ela não esteja bem otimizada, aumentando a perda ao longo do treinamento e prevendo apenas um rótulo para os dados do problema. Mesmo assim, foi interessante aprender sobre esse tipo de problema, aumentando meu conhecimento.

### Referências:

[1] CASSAR, Daniel. "ATP-303 NN 4.2 - Notebook MLP.ipynb". Material de Aula, 2025.

[2] Biblioteca Math. https://docs.python.org/3/library/math.html

[3] Biblioteca Random. https://docs.python.org/3/library/random.html

[4] Biblioteca Pandas. https://pandas.pydata.org/docs/user_guide/index.html#user-guide

[5] Biblioteca Seaborn. https://seaborn.pydata.org/

[6] Dataset estudado. https://archive.ics.uci.edu/dataset/53/iris

[7] Função de perda analisada. https://medium.com/ensina-ai/uma-explicação-visual-para-função-de-custo-binary-cross-entropy-ou-log-loss-eaee662c396c